In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import h5py

In [2]:
from data_augmentation import TransientMaker

In [3]:
###Definition:
path_to_original_data = '../track_02_training_data.h5'
start=9
stop=12
qntty_to_repeat = 50
name_dataset_to_be_created = '../dataset_test_realdata_augmented_from_RD_9_to_12.h5'

In [4]:
def get_random_transients():
        idx_transient_samples = np.random.randint(low=0,high=160,size=40)
        return idx_transient_samples

corrupted_fids = np.empty(((stop-start)*qntty_to_repeat,2048,2,40),dtype='complex128')
ppm_of_corrupted_fids = np.empty(((stop-start)*qntty_to_repeat,2048))
t_of_corrupted_fids = np.empty(((stop-start)*qntty_to_repeat,2048))
target_of_corrupted_fids = np.empty(((stop-start)*qntty_to_repeat,2048))

counter=0
for j in range(qntty_to_repeat):
    for i in range(start,stop):
        with h5py.File(path_to_original_data) as hf:
            fid = hf['transient_fids'][()][i:i+1]
            ppm = hf["ppm"][()][i:i+1]
            t = hf["t"][()][i:i+1]
            target = hf['target_spectra'][()][i:i+1]
    
        idx_transient_samples = get_random_transients()
        for k in range(len(idx_transient_samples)):
            corrupted_fids[counter,:,:,k] = fid[0,:,:,idx_transient_samples[k]]
        ppm_of_corrupted_fids[counter,:]=ppm[0,:]
        t_of_corrupted_fids[counter,:]=t[0,:]
        target_of_corrupted_fids[counter,:]=target[0,:]
        counter=counter+1


In [5]:
with h5py.File(name_dataset_to_be_created, 'w') as h5f:
    # Save inputs, targets, and labels into the HDF5 file
    h5f.create_dataset('transient_fids', data=corrupted_fids)
    h5f.create_dataset('ppm', data=ppm_of_corrupted_fids)
    h5f.create_dataset('t', data=t_of_corrupted_fids)
    h5f.create_dataset('target_spectra', data=target_of_corrupted_fids)

In [6]:
with h5py.File(name_dataset_to_be_created, 'r') as h5f:
    print(h5f.keys())
    print(h5f['transient_fids'][()].shape)
    print(h5f['ppm'][()].shape)
    print(h5f['target_spectra'][()].shape)
    print(h5f['t'][()].shape)

<KeysViewHDF5 ['ppm', 't', 'target_spectra', 'transient_fids']>
(150, 2048, 2, 40)
(150, 2048)
(150, 2048)
(150, 2048)
